In [ ]:
# Imports

# Libraries
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Local stuff
from branches import *
from panda_helpers import *
from helpers import *

In [ ]:
# Define the path to the file you want to open

# Make sure it is a __flat__ CAF file!!!
filename = "/pnfs/sbnd/persistent/sbndpro/mcp/mc/workshop/SBNWorkshop0421/prodoverlay_corsika_cosmics_proton_genie_nu_spill_gsimple-configf-v1_tpc/v09_19_00_01/caf/flat_caf_0-9f00feff-e742-419d-9856-9fe7428b93a9.root"

In [ ]:
# open it in uproot

tname = "recTree"
events = uproot.open(filename + ":" + tname)

In [ ]:
# Now we have a TTree!

events

In [ ]:
# Take some branches and load them into a pandas "data frame"

# What is a data frame?
# It is the way that pandas represents data. It works a lot like an excel spreedsheet or a SQL database.
# Each row of the data frame is a thing. That thing can have a number of different values, which are represented
# by columns.

# So, for example. We can make a neutrino data frame. Each row of the data frame is a neutrino. A neutrino
# has a bunch of different values like its Energy, interaction vertex, etc.

# mcbranches: a bunch of branch names that provide information on the neutrino interaction 
nudf = events.arrays(mcbranches, library="pd")

In [ ]:
# The data frame!!

# "entry" corresponds to the spill. And "subentry" corresponds to the index of the neutr
nudf

In [ ]:
# Make a plot!
_ = plt.hist(nudf["rec.mc.nu.E"])
plt.xlabel("Neutrino Energy [GeV]")
plt.ylabel("Entries")

In [ ]:
# How do we access more complex data?

# The CAF files have information we need to do event selection type analysis with. That is a lot of information. 
# Stuff like:
# -True neutrino interactions
# -True G4 particles
# -Reconstructed tracks
# -Reconstructed showers
# -CRT matching
# -PMT matching
# -Reco-to-Truth matching
# -...and more

# How do we handle this data in pandas, where every row is a thing?

# Two complexities:
# 1. We cannot combine (for example) reconstructed tracks and true neutrinos in the same data frame. Every row can
#    be a track, or every row can be a neutrino. But every row cannot be a track/neutrino thing.
#
#    So we are going to start out with separate data frame for each thing, and then build them together into a
#    single data frame that has all the information we need.
#
# 2. We need to handle nested data. A single spill might have multiple neutrinos. Or a single spill might have
#    multiple reconstructed slices, which have multiple reconstructed tracks, which have multiple truth-matched
#    paricles, etc.
# 
#    In C++, this is represented as a bunch of nested vectors. In pandas, we can use a __MultiIndex__. This is 
#    a very powerful pandas construct that lets us represent nested data

In [ ]:
# In order to handle nested data, I have a helper function that parses the FLAT CAF structure

# delete the old nudf
del nudf
nudf = loadbranches(events, mcbranches)
slcdf = loadbranches(events, slcbranches)
trkdf = loadbranches(events, trkbranches)

In [ ]:
# The neutrino data frame!
nudf

In [ ]:
# The slice data frame!
slcdf

In [ ]:
# The track data frame!
trkdf

In [ ]:
# Now, we are going to a quick bit of the numu Event selection.

# First, we can look at some reconstructed variables and compare them for neutrino-slices and cosmic-slices.
# To do this, we need to merge the neutrino data frame into the slice data frame using truth matching.

# We can do this with a "merge"

# Cut on the truth matching -- require the slice contains more than half of the deposited neutrino energy.
# This ensures that each neutrino can only have one reconstructed slice match
slc_has_nu_match = slcdf["rec.slc.tmatch.eff"] > 0.5

# Ignore index matches where the efficiency cut fails
slcdf.loc[np.invert(slc_has_nu_match) & (slcdf["rec.slc.tmatch.index"] >= 0), "rec.slc.tmatch.index"] = np.nan

matchdf = pd.merge(slcdf.reset_index(), # Merging can mess with the multi-index -- we'll fix this later
                 nudf.reset_index(),
                 left_on=["entry", "rec.slc.tmatch.index"], # Match on the entry number than the neutrino index
                 right_on=["entry", "rec.mc.nu..index"], 
                 how="left", # Keep every slice
                 )

matchdf = matchdf.set_index(["entry", "rec.slc..index"], verify_integrity=True)

# check that each neutrino matches to only one slice
assert(matchdf.groupby(["entry", "rec.mc.nu..index"])["rec.slc.charge"].count().max() == 1)

In [ ]:
matchdf

In [ ]:
# Energy of neutrinos with a matched slice!!

# NOTE: one thing to keep in mind -- merge's make a cut on the physics

_ = plt.hist(matchdf["rec.mc.nu.E"])

In [ ]:
# Now we can look at an example slice variable -- the Pandora "nu" score

var = matchdf["rec.slc.nu_score"]

is_numu_cc = (np.abs(matchdf["rec.mc.nu.pdg"]) == 14) & (matchdf["rec.mc.nu.iscc"])
is_cosmic = (matchdf["rec.slc.tmatch.index"] < 0)
bins=np.linspace(0, 1, 21)
_ = plt.hist(var[is_numu_cc], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
_ = plt.hist(var[is_cosmic], bins=bins, histtype="step", label="Cosmic")
_ = plt.legend()
_ = plt.xlabel(r"Pandora $\nu$ Score")

In [ ]:
# More complicated event selection stuff -- select a primary "Muon" track

# Using the algorithm from:
# https://sbn-docdb.fnal.gov/cgi-bin/private/RetrieveFile?docid=20139&filename=event_selection.pdf&version=2

# Now, using the trkdf
trkdf["trk_contained"] =\
    InFV(trkdf["rec.slc.reco.trk.start.x"], trkdf["rec.slc.reco.trk.start.y"], trkdf["rec.slc.reco.trk.start.z"]) &\
    InFV(trkdf["rec.slc.reco.trk.end.x"], trkdf["rec.slc.reco.trk.end.y"], trkdf["rec.slc.reco.trk.end.z"])

# check valid chi2 -- just look at collection plane for now
muon_chi2 = (trkdf["rec.slc.reco.trk.chi2pid2.chi2_muon"] < 30.) &\
    (trkdf["rec.slc.reco.trk.chi2pid2.chi2_proton"] > 60.)

# Valid primary track candidates
primary_track_candidate = (trkdf["trk_contained"] & muon_chi2 & (trkdf["rec.slc.reco.trk.len"] > 50.)) |\
        (trkdf["rec.slc.reco.trk.len"] > 100.)


primary_track = trkdf[primary_track_candidate]\
    .sort_values(["entry", "rec.slc..index", 'rec.slc.reco.trk.len'], ascending=[True, True, False])\
    .groupby(["entry", "rec.slc..index"]).head(1)

In [ ]:
# Now, merge the primary track into the slice df
df = pd.merge(matchdf.reset_index(), primary_track,
              left_on=["entry", "rec.slc..index"], # match on spill then slice number
              right_on=["entry", "rec.slc..index"],
              how="inner", # only keep slices with a primary track
              validate="one_to_one", # Always validate when you can! Don't put two primary tracks in a slice -- this would double-count a slice
             )

In [ ]:
# Now we can compute more stuff! Like the primary track momentum

is_numu_cc = (np.abs(df["rec.mc.nu.pdg"]) == 14) & (df["rec.mc.nu.iscc"])
is_cosmic = (df["rec.slc.tmatch.index"] < 0)

# Muon range momentum for contained tracks
var = df["rec.slc.reco.trk.rangeP.p_muon"]
bins = np.linspace(0, 2, 21)
_ = plt.hist(var[is_numu_cc & df["trk_contained"]], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
_ = plt.hist(var[is_cosmic & df["trk_contained"]], bins=bins, histtype="step", label="Cosmic")
plt.legend()
plt.xlabel("Reconstructed Muon Momentum (Contained) [GeV/c]")

In [ ]:
# And MCS for exiting tracks

var = df["rec.slc.reco.trk.mcsP.fwdP_muon"]
bins = np.linspace(0, 2, 21)
_ = plt.hist(var[is_numu_cc & np.invert(df["trk_contained"])], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
_ = plt.hist(var[is_cosmic & np.invert(df["trk_contained"])], bins=bins, histtype="step", label="Cosmic")
plt.legend()
plt.xlabel("Reconstructed Muon Momentum (Exiting) [GeV/c]")

In [ ]:
# Combine the two
recop = df["rec.slc.reco.trk.rangeP.p_muon"]+0. # copy
recop[np.invert(df["trk_contained"])] = df.loc[np.invert(df["trk_contained"]), "rec.slc.reco.trk.mcsP.fwdP_muon"]


var = recop
_ = plt.hist(var[is_numu_cc], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
_ = plt.hist(var[is_cosmic], bins=bins, histtype="step", label="Cosmic")
plt.legend()
plt.xlabel("Reconstructed Muon Momentum [GeV/c]")

In [ ]:
# Make some cuts!
passcut = (df["rec.slc.nu_score"] > 0.5) # Good nu score -- also rejects Clear-Cosmics (which have -1)

_ = plt.hist(var[is_numu_cc & passcut], bins=bins, histtype="step", label=r"$\nu_\mu$ CC")
_ = plt.hist(var[is_cosmic & passcut], bins=bins, histtype="step", label="Cosmic")
plt.legend()
plt.xlabel("Reconstructed Muon Momentum [GeV/c]")